In [1]:
#! /usr/bin/env python
"""
There is a small subset of glaciers for which surface temperature data was not available during the ablation season.
For these glaciers, we utilize a composite from all months to ensure we have complete glacier coverage.
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=False

In [2]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
        
def maskedarray_gt(data, value):
    """ Greater than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data > value] = value
    return data


def maskedarray_lt(data, value):
    """ Less than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data < value] = value
    return data


def ts_fromdebris_func(h, a, b, c):
    """ estimate surface temperature from debris thickness (h is debris thickness, a and k are coefficients) 
        Hill Equation"""
    return a * h**c / (b**c + h**c)


def debris_fromts_func(ts, a, b, c, hd_max=debris_prms.hd_max):
    """ estimate debris thickness from surface temperature (ts is surface temperature, a and k are coefficients) 
        Hill Equation"""
    # If temperature exceeds maximum of function cause NaN value
    max_value = ts_fromdebris_func(50, a, b, c)
    if ts.size == 1:
        if ts > max_value:
            ts = max_value
        if ts < 0:
            ts = 0
    else:
        ts[ts > a] = max_value
        ts[ts < 0] = 0
    # Debris thickness
    hd = (ts * b**c / (a - ts))**(1/c)
    return hd
        
    
def debris_fromts_maskedarray(ts_raw, a, b, c):
    """ Apply debris_fromts_func to masked array
        includes a mask of maximum values, since Michaelis-Mentin Equation has natural maximum 
    Parameters
    ----------
    ts_raw : np.ma.array
        masked array of the unmodified surface temperature
    Returns
    -------
    hd : np.ma.array 
        masked array of the debris thickness (m)
    """
    hd = debris_fromts_func(ts_raw.data, a, b, c)
    return hd

In [3]:
# ===== LOAD ALL GLACIERS =====
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)
dc_shp['CenLon_360'] = dc_shp['CenLon']
dc_shp.loc[dc_shp['CenLon_360'] < 0, 'CenLon_360'] = 360 + dc_shp.loc[dc_shp['CenLon_360'] < 0, 'CenLon_360']

# Add debris stats to area
dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area__1'].values))
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))

rgiid_list = [x.split('-')[1] for x in dc_shp['RGIId'].values]
main_glac_rgi_all = debris_prms.selectglaciersrgitable(rgiid_list)

# ===== DETERMINE ALL GLACIERS WITH AND WITHOUT OBSERVATIONS =====
hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/'

# Glaciers optimized
glac_tsopt_fns = []
rgiid_list_tsopt = []
for roi_extrap in debris_prms.roi_dict_extrap[debris_prms.roi]:
    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_extrap + '/'
    for i in os.listdir(hdopt_prms_fp):
        if i.endswith('_hdopt_prms.csv'):
            region = int(i.split('.')[0])
            if region in debris_prms.roi_rgidict[roi_extrap]:    
                rgiid_list_tsopt.append(i.split('_')[0])            
                glac_tsopt_fns.append(i)

glac_tsopt_fns = sorted(glac_tsopt_fns)
rgiid_list_tsopt = sorted(rgiid_list_tsopt)
main_glac_rgi_tsopt = debris_prms.selectglaciersrgitable(rgiid_list_tsopt)

# Glaciers extrapolated already
glac_tsopt_fns_wextrap = glac_tsopt_fns.copy()
rgiid_list_tsopt_wextrap = rgiid_list_tsopt.copy()
for roi_extrap in debris_prms.roi_dict_extrap[debris_prms.roi]:
    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_extrap + '/'
    
    for i in os.listdir(hdopt_prms_fp + '_extrap/'):
        if i.endswith('_hdopt_prms_extrap.csv'):
            region = int(i.split('.')[0])
            if region in debris_prms.roi_rgidict[roi_extrap]:    
                rgiid_list_tsopt_wextrap.append(i.split('_')[0])            
                glac_tsopt_fns_wextrap.append(i)

glac_tsopt_fns_wextrap = sorted(glac_tsopt_fns_wextrap)
rgiid_list_tsopt_wextrap = sorted(rgiid_list_tsopt_wextrap)
main_glac_rgi_tsopt_wextrap = debris_prms.selectglaciersrgitable(rgiid_list_tsopt_wextrap)

main_glac_rgi_all['DC_Area_%'] = main_glac_rgi_all.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_tsopt['DC_Area_%'] = main_glac_rgi_tsopt.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_all['DC_Area_v2'] = main_glac_rgi_all['Area'] * main_glac_rgi_all['DC_Area_%'] / 100
main_glac_rgi_tsopt['DC_Area_v2'] = main_glac_rgi_tsopt['Area'] * main_glac_rgi_tsopt['DC_Area_%'] / 100

# Glaciers lacking optimization
rgiids_missing = set(main_glac_rgi_all.rgino_str.values) - set(main_glac_rgi_tsopt_wextrap.rgino_str.values)
rgiids_missing = sorted(rgiids_missing)
if len(rgiids_missing) > 0:    
    main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)
    missing_idx = [rgiid_list.index(x) for x in rgiids_missing]
    main_glac_rgi_missing['DC_Area_%'] = main_glac_rgi_missing.RGIId.map(dc_areaperc_dict).fillna(0)
    main_glac_rgi_missing['DC_Area_v2'] = main_glac_rgi_missing['Area'] * main_glac_rgi_missing['DC_Area_%'] / 100
    main_glac_rgi_missing.reset_index(inplace=True, drop=True)
    
else:
    print('\n\n NO MISSING GLACIERS FOR REGION', debris_prms.roi)

1965 glaciers in region 3 are included in this model run: ['00093', '00094', '00095', '00096', '00097', '00098', '00099', '00100', '00102', '00103', '00104', '00105', '00106', '00107', '00108', '00109', '00110', '00113', '00114', '00115', '00116', '00117', '00118', '00120', '00123', '00124', '00125', '00126', '00128', '00129', '00130', '00131', '00132', '00133', '00134', '00135', '00136', '00137', '00139', '00140', '00141', '00142', '00143', '00144', '00145', '00146', '00147', '00148', '00150', '00151'] and more
This study is focusing on 1965 glaciers in region [3]
75 glaciers in region 3 are included in this model run: ['00527', '00718', '00923', '00928', '00937', '00945', '00987', '01005', '01192', '01210', '01341', '01354', '01476', '01479', '01586', '01603', '01616', '01669', '01675', '01701', '01705', '01739', '01756', '01885', '01889', '01891', '01907', '02143', '02147', '02149', '02168', '02184', '02260', '02268', '02272', '02273', '02282', '02295', '02296', '02298', '02300', '0

In [4]:
print('\n', debris_prms.roi + ': calibration includes', main_glac_rgi_tsopt.shape[0], 'glaciers covering',
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum(),1)), 'km2 (' + 
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum() / main_glac_rgi_all['DC_Area_v2'].sum() * 100,1)) + 
      '%) of the total debris-covered glacier area\n')    

missing_idx = [rgiid_list.index(x) for x in rgiids_missing]
print('\n', debris_prms.roi + ': missing includes', main_glac_rgi_missing.shape[0], 'glaciers covering',
      str(np.round(main_glac_rgi_all.loc[missing_idx,'DC_Area_v2'].sum(),1)), 'km2 (' + 
      str(np.round(main_glac_rgi_all.loc[missing_idx,'DC_Area_v2'].sum() / main_glac_rgi_all['DC_Area_v2'].sum() * 100,1)) + 
      '%) of the total debris-covered glacier area\n')    

print('  glaciers all:', main_glac_rgi_all['DC_Area_v2'].shape[0], 
      'total dc area [km2]:', np.round(main_glac_rgi_all['DC_Area_v2'].sum(),1))


 03: calibration includes 75 glaciers covering 182.2 km2 (20.8%) of the total debris-covered glacier area


 03: missing includes 149 glaciers covering 165.9 km2 (18.9%) of the total debris-covered glacier area

  glaciers all: 1965 total dc area [km2]: 875.8


In [5]:
A = main_glac_rgi_missing.copy()
print(A.columns)
A = A.sort_values('DC_Area_v2', ascending=False)
A.reset_index(inplace=True, drop=True)
print(A['DC_Area_v2'].sum())
print(A.loc[0:10,'DC_Area_v2'].sum())
A.loc[0:10,:]

Index(['O1Index', 'RGIId', 'CenLon', 'CenLat', 'O1Region', 'O2Region', 'Area',
       'Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax', 'Form', 'TermType',
       'Surging', 'RefDate', 'glacno', 'rgino_str', 'RGIId_float', 'DC_Area_%',
       'DC_Area_v2'],
      dtype='object')
165.85886545
94.05595543000001


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,DC_Area_%,DC_Area_v2
0,2902,RGI60-03.02904,-72.4674,82.3974,3,1,1384.541,0,2107,1227,...,56860,0,1,1,20019999,2904,03.02904,3.02904,2.878,39.847090
1,2847,RGI60-03.02849,-78.9149,82.2671,3,1,1078.334,0,2012,1111,...,77014,0,1,0,20019999,2849,03.02849,3.02849,0.959,10.341223
2,2666,RGI60-03.02668,-75.5681,82.1571,3,1,1734.500,0,2346,1463,...,75434,0,1,1,20019999,2668,03.02668,3.02668,0.507,8.793915
3,122,RGI60-03.00123,-76.5299,80.3547,3,3,1594.482,0,2106,1438,...,74478,0,1,2,20019999,123,03.00123,3.00123,0.365,5.819859
4,2685,RGI60-03.02687,-77.8410,81.9358,3,1,875.281,0,2228,1347,...,67794,0,1,1,20019999,2687,03.02687,3.02687,0.622,5.444248
5,2465,RGI60-03.02467,-81.5122,75.0577,3,6,2622.813,0,1921,969,...,82582,1,1,1,19999999,2467,03.02467,3.02467,0.179,4.694835
6,112,RGI60-03.00113,-75.3588,80.5373,3,3,999.861,0,2074,1446,...,71606,1,1,0,20019999,113,03.00113,3.00113,0.437,4.369393
7,1515,RGI60-03.01517,-80.0780,76.8310,3,5,1741.310,0,1348,528,...,56818,0,1,3,20019999,1517,03.01517,3.01517,0.236,4.109492
8,3387,RGI60-03.03389,-80.3171,81.6622,3,1,774.565,0,1887,1476,...,702,0,1,0,20019999,3389,03.03389,3.03389,0.503,3.896062
9,2834,RGI60-03.02836,-78.7131,82.1353,3,1,660.666,0,2282,1162,...,62175,0,1,1,20019999,2836,03.02836,3.02836,0.520,3.435463


In [6]:
# # print('\nHACK TO EXTRAPOLATE TO CALIBRATED GLACIERS FOR COMPARISON\n')
# rgiids_missing = ['15.02167']
# # # rgiids_missing = ['2.14297']
# # # rgiids_missing = ['11.02472']
# # # rgiids_missing = ['12.01132']
# # # rgiids_missing = ['13.43165']
# # # rgiids_missing = ['15.03473']
# # # rgiids_missing = ['15.04045']
# # # rgiids_missing = ['18.02397']
# rgiids_missing = ['15.02231']
# main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)

In [7]:
extrap_uncalibrated_glaciers = True
overwrite_hd = False

hd_fp = debris_prms.hd_fp + 'extrap/'
if not os.path.exists(hd_fp):
    os.makedirs(hd_fp)
    
mf_fp = hd_fp + 'meltfactor/'
if not os.path.exists(mf_fp):
    os.makedirs(mf_fp)
    
fig_extrap = debris_prms.output_fig_fp + debris_prms.roi + '/' + 'extrap/'
if not os.path.exists(fig_extrap):
    os.makedirs(fig_extrap)

if extrap_uncalibrated_glaciers:
    
    # ===== NEAREST GLACIERS WITH DATA =====
    n_glac_nearest = 1000
    if n_glac_nearest > main_glac_rgi_tsopt.shape[0]:
        n_glac_nearest = main_glac_rgi_tsopt.shape[0]

    nearest_dict = {}
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[0]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        if glac_idx%500 == 0:
            print(glac_idx, glac_str)
        latlon_dist = (((main_glac_rgi_tsopt['CenLat'].values - main_glac_rgi_missing['CenLat'].values[glac_idx])**2 + 
                        (main_glac_rgi_tsopt['CenLon'].values - main_glac_rgi_missing['CenLon'].values[glac_idx])**2)**0.5)

        latlon_nearidx_list = np.argsort(latlon_dist)[0:n_glac_nearest]
        rgiid_nearest_list = list(main_glac_rgi_tsopt.loc[latlon_nearidx_list,'rgino_str'].values)

        nearest_dict[glac_str] = rgiid_nearest_list

    # Ts filenames
    ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values[4290:]):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[0]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        rgiid = main_glac_rgi_missing.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        # Ts filename
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        
        # Hd filename
        hd_fn = debris_prms.hd_fn_sample.replace('XXXX', glac_str_noleadzero).replace('.tif','_extrap.tif')

        # Ice thickness filenames
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        # Record values from nearest for the melt factors after debris thickness is extrapolated
        melt_2cm_nearest = None
        melt_cleanice_nearest = None
        func_coeff_nearest = None
        
        if ((not os.path.exists(hd_fp + hd_fn) or overwrite_hd) and os.path.exists(thick_dir + thick_fn) and 
            ts_fn not in ['0.0']):
            
            print(glac_idx, glac_str)
            
            # Create glacier feature
            gf = create_glacfeat(thick_dir, thick_fn)
        
            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if not os.path.exists(dc_shp_proj_fn):
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + 
                                            debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + 
                                                    str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_ts=True, ts_fn=ts_fn, gf_add_slope_aspect=False)

            # ===== PLOTS =====
            if debug:
                # DEM
                var_full2plot = gf.z1.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                # Surface temperature
                var_full2plot = gf.ts.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
                # Surface temperature (debris-covered)
                var_full2plot = gf.ts.copy()
                var_full2plot.mask = gf.dc_mask
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
            

            # ===== SURFACE TEMPERATURE FOR THINNEST DEBRIS (connected to terminus) =====
            outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)   
            bin_idx_dc = np.where(outbins_df['dc_bin_count_valid'] > 0)[0]
            bin_idx_dif = list(bin_idx_dc[1:] - bin_idx_dc[:-1])
            if not(np.sum(bin_idx_dif) == len(bin_idx_dc)-1):
                idx_jumpinbins = bin_idx_dif.index(next(filter(lambda x: x>1, bin_idx_dif)))
                bin_idx_dc = bin_idx_dc[0:idx_jumpinbins+1]
            
            ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            
            if debug:
                print('ts_min:', np.round(ts_min,1), ' ts_max:', np.round(ts_max,1))
            
            if np.isnan(ts_min) and np.isnan(ts_max):
                troubleshoot_fp = (debris_prms.output_fp + 'errors/no_Ts_data-extrap/' + debris_prms.roi + '/')
                if not os.path.exists(troubleshoot_fp):
                    os.makedirs(troubleshoot_fp)
                txt_fn_extrapfail = glac_str + "-noTs-extrap.txt"
                with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                    text_file.write(glac_str + ' no surface temperature data but made it past preprocessing to extrap')

            else:
                # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                # Load parameters from nearest neighbor
                rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                n_nearest = 0
                n_success = 0
                min_n_nearest = 10
                hd_ts_list = []
                mf_list = []
                while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                    rgi_str_nearest = rgiid_nearest_list[n_nearest]
                    if rgi_str_nearest.startswith('0'):
                        rgi_str_nearest = rgi_str_nearest[1:]
#                     if debug:
#                         print(n_nearest, 'rgi nearest:', rgi_str_nearest)

                    # Load parameters
                    df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                    roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                    if roi_nearest in ['13','14','15']:
                        roi_nearest = 'HMA'
                    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                    df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                    melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                    melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                    func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                    func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]
                    
                    if melt_2cm_nearest is None:
                        melt_2cm_nearest = melt_2cm.copy()
                        melt_cleanice_nearest = melt_cleanice.copy()
                        func_coeff_nearest = func_coeff.copy()

                    # Estimate debris thickness of thinnest bin
                    hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    
                    if debug:
                        print('     thin:', np.round(ts_min,1), np.round(hd_thin,3), 'm', 
                              '    thick:', np.round(ts_max,1), np.round(hd_thick,3), 'm')

    #                 print(func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2], ts_min, hd_thin)

                    # Minimum and maximum debris thickness are reasonable
                    if hd_thin > 0 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                        if debug:
                            print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                        hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                        hd_array[hd_array<0] = 0

                        hd_ts_list.append(hd_array)
                        n_success += 1

                    n_nearest += 1
                    
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd1/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd1.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in first round')
                    
                    
                # ===== SECOND ROUND: NEGATIVE VALUES CAUSE HD_THIN TO BE 0 =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_min < 0:
                    
                    if debug:
                        print('\n-----\nROUND 2 OF EXTRAPOLATION')
                    
                    ts_offset = abs(ts_min)
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
#                             ts_array = gf.ts.data.copy() + ts_offset
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                            if roi_nearest in ['13','14','15']:
                                roi_nearest = 'HMA'
                            hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1
                        ts_offset = 1
                        
                        
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd2/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd2.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in second round')
                    
                # ===== THIRD ROUND: ASSUME TOO POSITIVE CAUSING HD TO BE VERY THICK =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_max > 20:
                    
                    if debug:
                        print('\n-----\nROUND 3 OF EXTRAPOLATION')
                    
                    ts_offset = -1
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            roi_nearest = str(int(rgi_str_nearest.split('.')[0])).zfill(2)
                            if roi_nearest in ['13','14','15']:
                                roi_nearest = 'HMA'
                            hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_nearest + '/'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1

                # ===== ESTIMATE DEBRIS THICKNESS FROM ALL COMBINATIONS =====
                if len(hd_ts_list) > 0:
                    # DEBRIS THICKNESS based on median of the plausible nearest values
                    hd_ts_all = np.array(hd_ts_list)
                    hd_ts_med = np.median(hd_ts_all, axis=0)
                    gf.debris_thick_ts = np.ma.array(hd_ts_med, mask=gf.dc_mask)

                    if debug:
                        close_fig=False
                    else:
                        close_fig=True

                    # Debris thickness
                    var_full2plot = gf.debris_thick_ts.copy()
                    clim = (0,1)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' hd (from ts)'], 'inferno', 'hd (m)', 
                               fn=fig_extrap + gf.feat_fn +'_hd_ts.png', close_fig=close_fig)

                    # ===== EXPORT DEBRIS THICKNESS AND MELT FACTOR TIFS ===== 
                    # Debris thickness
                    gf.debris_thick_ts.mask = gf.dc_mask
                    iolib.writeGTiff(gf.debris_thick_ts, hd_fp + hd_fn, gf.ds_dict['z1'])

                    # Optimized parameters from nearest glacier for melt factor
                    hdopt_cns = ['glac_str', 'melt_mwea_clean', 'melt_mwea_2cm', 'b0', 'k']
                    df_hdopt_prms = pd.DataFrame(np.zeros((1,len(hdopt_cns))), columns=hdopt_cns)
                    df_hdopt_prms['glac_str'] = glac_str
                    df_hdopt_prms.loc[0,'melt_mwea_clean'] = melt_cleanice_nearest
                    df_hdopt_prms.loc[0,'melt_mwea_2cm'] = melt_2cm_nearest
                    df_hdopt_prms.loc[0,'b0'] = func_coeff_nearest[0]
                    df_hdopt_prms.loc[0,'k'] = func_coeff_nearest[1]
                    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/_extrap/'
                    if not os.path.exists(hdopt_prms_fp):
                        os.makedirs(hdopt_prms_fp)
                    df_hdopt_prms.to_csv(hdopt_prms_fp + glac_str + '_hdopt_prms_extrap.csv', index=False)
                    
                    # Melt factor
                    gf.meltfactor_ts = (
                        melt_fromdebris_func(gf.debris_thick_ts, func_coeff_nearest[0], func_coeff_nearest[1]) 
                        / melt_cleanice_nearest)
                    # limit melt rates to modeled 2 cm rate
                    gf.meltfactor_ts = np.ma.array(
                        maskedarray_gt(gf.meltfactor_ts, melt_2cm_nearest / melt_cleanice_nearest), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Linearly interpolate between 0 cm and 2 cm for the melt rate
                    def meltfactor_0to2cm_adjustment(mf, melt_clean, melt_2cm, hd):
                        """ Linearly interpolate melt factors between 0 and 2 cm 
                            based on clean ice and 2 cm sub-debris melt """
                        mf = np.nan_to_num(mf,0)
                        mf[(hd >= 0) & (hd < 0.02)] = (
                            1 + hd[(hd >= 0) & (hd < 0.02)] / 0.02 * (melt_2cm - melt_clean) / melt_clean)
                        return mf
                    gf.meltfactor_ts = np.ma.array(
                        meltfactor_0to2cm_adjustment(gf.meltfactor_ts.data.copy(), melt_cleanice_nearest, 
                                                     melt_2cm_nearest, gf.debris_thick_ts.data), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Plot melt factor
                    var_full2plot = gf.meltfactor_ts.copy()
                    clim = (0,1.25)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' melt factor'], 'inferno', 'melt factor (-)', 
                               fn=fig_extrap + gf.feat_fn +'_mf.png', close_fig=True)

                    gf.meltfactor_ts.mask = gf.dc_mask
                    mf_fn = debris_prms.mf_fn_sample.replace('XXXX',gf.glacnum).replace('.tif','_extrap.tif')
                    iolib.writeGTiff(gf.meltfactor_ts, mf_fp + mf_fn, gf.ds_dict['z1'])


                    # ===== EXPORT THE BINNED DEBRIS THICKNESS AND MELT FACTOR =====
                    # Output debris thickness
                    outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                    hd_extrap_bin_fp = debris_prms.mb_binned_fp_wdebris_hdts + '../_wdebris_hdts_extrap/'
                    if not os.path.exists(hd_extrap_bin_fp):
                        os.makedirs(hd_extrap_bin_fp)
                    outbins_df.to_csv(hd_extrap_bin_fp + glac_str + '_mb_bins_hdts_extrap.csv', index=False)

                else:
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates')

0 03.00108
0 03.00108


/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:118: RuntimeWarning: All-NaN slice encountered
/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:119: RuntimeWarning: All-NaN slice encountered


1 03.00113


/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:118: RuntimeWarning: All-NaN slice encountered
/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:119: RuntimeWarning: All-NaN slice encountered


2 03.00123


...

/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:118: RuntimeWarning: All-NaN slice encountered
/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:119: RuntimeWarning: All-NaN slice encountered


147 03.04384


/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:118: RuntimeWarning: All-NaN slice encountered
/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:119: RuntimeWarning: All-NaN slice encountered


148 03.04385


/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:118: RuntimeWarning: All-NaN slice encountered
/Users/drounce/opt/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:119: RuntimeWarning: All-NaN slice encountered


In [8]:
# bin_idx_dc_thin = bin_idx_dc[-1]
# outbins_df.loc[bin_idx_dc_thin, 'hd_ts_med_m']
# outbins_df.loc[:,['bin_center_elev_m', 'dc_ts_med', 'hd_ts_med_m']]

In [9]:
print('\nDONE!\n')


DONE!

